# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
!pip install nltk

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# import libraries
from sqlalchemy import create_engine 
import nltk
import pandas as pd
import sqlite3
import pickle

import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [5]:
# load data from database
engine = create_engine('sqlite:///Disaster_Analysis.db')
df = pd.read_sql_table('Messages_with_Cats', engine)

# Choose the X and Y (feature and target variables) for ML Model
X = df['message']
Y = df.iloc[:,4:]


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    '''
    INPUT 
        text: Text to be processed   
    OUTPUT
        Returns an array of words after being processed; by being tokenized, lower cased, stripped, and lemmatized
    '''
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    '''
    Define a custom estimator that extract the feature from a sentence checking if the text is starting with a verb
    
    '''

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [8]:
def model_pipeline():
    
    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ]))
            ])),
        
            ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])
    return pipeline

In [24]:
def model_pipeline_improved():
    
    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ])),

                ('starting_verb', StartingVerbExtractor())
            ])),

            ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=10)))
        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Define the model and fit the training dataset
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
# Predict from trained model using the test dataset
y_pred = model.predict(X_test)

In [12]:
print(classification_report(y_test.values, y_pred,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      5069
               request       0.70      0.49      0.58      1140
                 offer       0.00      0.00      0.00        31
           aid_related       0.66      0.52      0.58      2756
          medical_help       0.30      0.17      0.22       554
      medical_products       0.26      0.21      0.23       357
     search_and_rescue       0.56      0.05      0.09       192
              security       0.17      0.01      0.02       114
              military       0.47      0.07      0.12       210
           child_alone       0.00      0.00      0.00         0
                 water       0.43      0.33      0.38       412
                  food       0.43      0.38      0.40       722
               shelter       0.37      0.31      0.34       583
              clothing       0.61      0.21      0.31        97
                 money       0.35      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'clf__estimator__n_neighbors': [3, 7, 10],
    'clf__estimator__weights': ['uniform', 'distance']
}

# Create new model with same pipeline and apply grid search on it
model2 = model_pipeline()

cv = GridSearchCV(model2, param_grid = parameters, cv=2, scoring='accuracy', verbose=10)

In [12]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__features', 'estimator__clf', 'estimator__features__n_jobs', 'estimator__features__transformer_list', 'estimator__features__transformer_weights', 'estimator__features__text_pipeline', 'estimator__features__text_pipeline__memory', 'estimator__features__text_pipeline__steps', 'estimator__features__text_pipeline__vect', 'estimator__features__text_pipeline__tfidf', 'estimator__features__text_pipeline__vect__analyzer', 'estimator__features__text_pipeline__vect__binary', 'estimator__features__text_pipeline__vect__decode_error', 'estimator__features__text_pipeline__vect__dtype', 'estimator__features__text_pipeline__vect__encoding', 'estimator__features__text_pipeline__vect__input', 'estimator__features__text_pipeline__vect__lowercase', 'estimator__features__text_pipeline__vect__max_df', 'estimator__features__text_pipeline__vect__max_features', 'estimator__features__text_pipeline__vect__min_df', 'estimator__fea

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.fit(X_train, y_train)
#cv.get_params()

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] clf__estimator__n_neighbors=3, clf__estimator__weights=uniform ..
[CV]  clf__estimator__n_neighbors=3, clf__estimator__weights=uniform, score=0.1930627606550707, total= 3.6min
[CV] clf__estimator__n_neighbors=3, clf__estimator__weights=uniform ..


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.1min remaining:    0.0s


[CV]  clf__estimator__n_neighbors=3, clf__estimator__weights=uniform, score=0.21414038657171922, total= 3.6min
[CV] clf__estimator__n_neighbors=3, clf__estimator__weights=distance .


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 14.1min remaining:    0.0s


[CV]  clf__estimator__n_neighbors=3, clf__estimator__weights=distance, score=0.19296104160309227, total= 3.4min
[CV] clf__estimator__n_neighbors=3, clf__estimator__weights=distance .


...

[CV]  clf__estimator__n_neighbors=10, clf__estimator__weights=uniform, score=0.20750686603600854, total= 3.9min
[CV] clf__estimator__n_neighbors=10, clf__estimator__weights=uniform .


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 65.7min remaining:    0.0s


[CV]  clf__estimator__n_neighbors=10, clf__estimator__weights=uniform, score=0.21627670396744658, total= 4.1min
[CV] clf__estimator__n_neighbors=10, clf__estimator__weights=distance 
[CV]  clf__estimator__n_neighbors=10, clf__estimator__weights=distance, score=0.19580917505848847, total= 3.9min
[CV] clf__estimator__n_neighbors=10, clf__estimator__weights=distance 
[CV]  clf__estimator__n_neighbors=10, clf__estimator__weights=distance, score=0.20742624618514752, total= 3.8min


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 88.8min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_neighbors': [3, 7, 10], 'clf__estimator__weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=10)

In [14]:
y_pred2 = cv.predict(X_test)

In [19]:
best_clf = cv.best_estimator_
best_clf

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ic_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform'),
           n_jobs=1))])

In [17]:
print(classification_report(y_test.values, y_pred2,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.93      0.88      5075
               request       0.76      0.49      0.59      1124
                 offer       0.00      0.00      0.00        30
           aid_related       0.67      0.56      0.61      2706
          medical_help       0.29      0.13      0.18       553
      medical_products       0.24      0.17      0.20       351
     search_and_rescue       0.83      0.03      0.05       192
              security       0.00      0.00      0.00       124
              military       0.67      0.05      0.09       203
           child_alone       0.00      0.00      0.00         0
                 water       0.51      0.30      0.38       425
                  food       0.49      0.41      0.45       757
               shelter       0.34      0.25      0.29       571
              clothing       0.85      0.16      0.27       107
                 money       0.64      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
model_improved = model_pipeline_improved()
model_improved.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ic_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform'),
           n_jobs=1))])

In [26]:
y_pred3 = model_improved.predict(X_test)

In [27]:
print(classification_report(y_test.values, y_pred3,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5075
               request       0.77      0.50      0.61      1124
                 offer       0.00      0.00      0.00        30
           aid_related       0.67      0.56      0.61      2706
          medical_help       0.35      0.12      0.17       553
      medical_products       0.33      0.15      0.21       351
     search_and_rescue       0.75      0.03      0.06       192
              security       0.00      0.00      0.00       124
              military       0.67      0.05      0.09       203
           child_alone       0.00      0.00      0.00         0
                 water       0.62      0.27      0.37       425
                  food       0.53      0.39      0.45       757
               shelter       0.36      0.21      0.26       571
              clothing       0.93      0.13      0.23       107
                 money       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [29]:
model_filepath = 'classifier.pk'
file = open(model_filepath, 'wb')
pickle.dump(model, file)
file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.